In [ ]:
import pandas as pd
import numpy as np

import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, LeakyReLU, Input, concatenate
from keras.callbacks import EarlyStopping
from keras.optimizers import Adam, RMSprop
from keras.regularizers import l1_l2


import glob
import os

from scipy.stats import zscore

In [ ]:
weather_path = os.path.join('..','data','weather','precip_temp.csv')
weather_df = pd.read_csv(weather_path)
clients_path = os.path.join('..','data','wifi','**','Clients per day.csv')
clients_df = pd.concat(map(lambda csv: pd.read_csv(csv, parse_dates=[0]),
                           glob.glob(clients_path)), ignore_index=True)
weekday_labels = []
for d in range(7):
    label = 'weekday-%i' % d
    weekday_labels.append(label)
    clients_df[label] = [int(dt.weekday()==d) for dt in clients_df['Time']]
# Put clients date in the same format as it is in the weather data
clients_df['Time'] = [dt.strftime('%Y-%m-%d') for dt in clients_df['Time']]

# This merge ignores the missing data point in the weather data
all_data = clients_df.merge(weather_df, left_on='Time', right_on='DATE')

cols_to_norm = ['TMIN', 'TMAX', 'PRCP']
for col in cols_to_norm:
    all_data[col] = zscore(all_data[col])

x_labels = weekday_labels + ['PRCP', 'TMAX', 'TMIN']
day_x=all_data[weekday_labels].values
weather_x=all_data[['PRCP', 'TMAX', 'TMIN']].values
date_x=np.expand_dims(np.arange(0,353/365,1/365).astype('float32'),axis=1)
y_labels = ['# Clients']
x = all_data[x_labels].values
y = all_data[y_labels].values
all_data

In [ ]:
epochs = 1000
reg=None#l1_l2(0.01,0.02)

def stackLayers(layerSet):
    #print (layerSet)
    stack = layerSet[0]
    if len(layerSet)>1:
        for i in range (0,len(layerSet)-1):
            stack=layerSet[i+1](stack)
    return (stack)
weatherLayers=[]
weatherLayers+=[Input((3,))]
weatherLayers+=[Dense(10, activation='relu',kernel_regularizer=reg)]
weatherLayers+=[Dropout(0.5)]
weatherLayers+=[Dense(10, activation='relu',kernel_regularizer=reg)]
weatherLayers+=[Dropout(0.5)]
weatherLayers+=[Dense(5, activation='relu',kernel_regularizer=reg)]
weatherLayers+=[Dropout(0.5)]
weatherLayers+=[Dense(4, activation='relu',kernel_regularizer=reg)]
weatherOut=stackLayers(weatherLayers)
print (weatherOut)

dayLayers=[]
dayLayers+=[Input((7,))]
dayLayers+=[Dense(10, activation='relu',kernel_regularizer=reg)]
dayLayers+=[Dropout(0.5)]
dayLayers+=[Dense(10, activation='relu',kernel_regularizer=reg)]
dayLayers+=[Dropout(0.5)]
dayLayers+=[Dense(10, activation='relu',kernel_regularizer=reg)]
dayLayers+=[Dropout(0.5)]
dayLayers+=[Dense(5, activation='relu',kernel_regularizer=reg)]
dayOut=stackLayers(dayLayers)

dateLayers=[]
dateLayers+=[Input((1,))]
dateLayers+=[Dense(10, activation='relu',kernel_regularizer=reg)]
dateLayers+=[Dropout(0.5)]
dateLayers+=[Dense(10, activation='relu',kernel_regularizer=reg)]
dateLayers+=[Dropout(0.5)]
dateLayers+=[Dense(10, activation='relu',kernel_regularizer=reg)]
dateLayers+=[Dropout(0.5)]
dateLayers+=[Dense(5, activation='relu',kernel_regularizer=reg)]
dateOut=stackLayers(dateLayers)

mainLayers=[]
mainLayers+=[concatenate([weatherOut,dayOut,dateOut])]
mainLayers+=[Dense(10, activation='relu',kernel_regularizer=reg)]
mainLayers+=[Dropout(0.5)]
mainLayers+=[Dense(10, activation='relu',kernel_regularizer=reg)]
mainLayers+=[Dropout(0.5)]
mainLayers+=[Dense(10, activation='relu',kernel_regularizer=reg)]
mainLayers+=[Dropout(0.5)]
mainLayers+=[Dense(1, activation='linear')]
mainOut=stackLayers(mainLayers)

model=Model(inputs=[weatherLayers[0],dayLayers[0],dateLayers[0]],outputs=[mainOut])
model.compile(loss='mean_absolute_percentage_error',
              optimizer=Adam(0.0001))

model.summary()

early_stopping = EarlyStopping(patience=50, verbose=1)
history = model.fit([weather_x,day_x,date_x], [y],
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[early_stopping])
from math import ceil



In [ ]:

evalSeg=317
print (weather_x[evalSeg:].shape)
model.evaluate(x=[weather_x[evalSeg:],day_x[evalSeg:],date_x[evalSeg:]],y=[y[evalSeg:]])

In [ ]:
y